In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
import time
!pip install missingno
import missingno as msno
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('swiggy_imputed_data.csv',dtype={'is_weekend':'int'})
df1 = pd.read_csv('swiggy_cleaned_data.csv',dtype={'is_weekend':'int'})
df.head()

,age,ratings,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,time_taken,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time,order_time_of_day,distance
0,37.0,4.9,sunny,high,2,snack,motorcycle,0,no,urban,24.0,INDO,19,3,Saturday,1,15.0,morning,3.03
1,34.0,4.5,stormy,jam,2,snack,scooter,1,no,metropolitian,33.0,BANG,25,3,Friday,0,5.0,night,20.18
2,23.0,4.4,sandstorms,low,0,drinks,motorcycle,1,no,urban,26.0,BANG,19,3,Saturday,1,15.0,morning,1.55
3,38.0,4.7,sunny,medium,0,buffet,motorcycle,1,no,metropolitian,21.0,COIMB,5,4,Tuesday,0,10.0,evening,7.79
4,32.0,4.6,cloudy,high,1,snack,scooter,1,no,metropolitian,30.0,CHEN,26,3,Saturday,1,15.0,afternoon,6.21


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45073 entries, 0 to 45072
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  45073 non-null  float64
 1   ratings              45073 non-null  float64
 2   weather              45073 non-null  object 
 3   traffic              45073 non-null  object 
 4   vehicle_condition    45073 non-null  int64  
 5   type_of_order        45073 non-null  object 
 6   type_of_vehicle      45073 non-null  object 
 7   multiple_deliveries  45073 non-null  int64  
 8   festival             45073 non-null  object 
 9   city_type            45073 non-null  object 
 10  time_taken           45073 non-null  float64
 11  city_name            45073 non-null  object 
 12  order_day            45073 non-null  int64  
 13  order_month          45073 non-null  int64  
 14  order_day_of_week    45073 non-null  object 
 15  is_weekend           45073 non-null 

In [4]:
df.shape

(45073, 19)

In [5]:
df.columns

Index(['age', 'ratings', 'weather', 'traffic', 'vehicle_condition',
       'type_of_order', 'type_of_vehicle', 'multiple_deliveries', 'festival',
       'city_type', 'time_taken', 'city_name', 'order_day', 'order_month',
       'order_day_of_week', 'is_weekend', 'pickup_time', 'order_time_of_day',
       'distance'],
      dtype='object')

In [6]:
df.corr(numeric_only=True)['time_taken'].sort_values(ascending=False)


time_taken             1.000000
multiple_deliveries    0.378074
age                    0.306066
distance               0.295965
order_day              0.023535
is_weekend            -0.003126
pickup_time           -0.008528
order_month           -0.011739
vehicle_condition     -0.244269
ratings               -0.362069
Name: time_taken, dtype: float64

In [7]:
temp_df = df.copy()
X = temp_df.drop(columns='time_taken')
y = temp_df['time_taken']

X.head()

,age,ratings,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time,order_time_of_day,distance
0,37.0,4.9,sunny,high,2,snack,motorcycle,0,no,urban,INDO,19,3,Saturday,1,15.0,morning,3.03
1,34.0,4.5,stormy,jam,2,snack,scooter,1,no,metropolitian,BANG,25,3,Friday,0,5.0,night,20.18
2,23.0,4.4,sandstorms,low,0,drinks,motorcycle,1,no,urban,BANG,19,3,Saturday,1,15.0,morning,1.55
3,38.0,4.7,sunny,medium,0,buffet,motorcycle,1,no,metropolitian,COIMB,5,4,Tuesday,0,10.0,evening,7.79
4,32.0,4.6,cloudy,high,1,snack,scooter,1,no,metropolitian,CHEN,26,3,Saturday,1,15.0,afternoon,6.21


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (36058, 18)
The shape of test data is (9015, 18)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45073 entries, 0 to 45072
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  45073 non-null  float64
 1   ratings              45073 non-null  float64
 2   weather              45073 non-null  object 
 3   traffic              45073 non-null  object 
 4   vehicle_condition    45073 non-null  int64  
 5   type_of_order        45073 non-null  object 
 6   type_of_vehicle      45073 non-null  object 
 7   multiple_deliveries  45073 non-null  int64  
 8   festival             45073 non-null  object 
 9   city_type            45073 non-null  object 
 10  time_taken           45073 non-null  float64
 11  city_name            45073 non-null  object 
 12  order_day            45073 non-null  int64  
 13  order_month          45073 non-null  int64  
 14  order_day_of_week    45073 non-null  object 
 15  is_weekend           45073 non-null 

In [10]:
num_cols = [ "age", "ratings", "pickup_time", "distance"]
    
nominal_cat_cols = [ "weather","type_of_order","type_of_vehicle","festival","city_name","order_day", 
                    "order_month","order_day_of_week"] # "multiple_deliveries","vehicle_condition"

ordinal_cat_cols = ["traffic",'city_type','order_time_of_day']         



In [11]:
print(df['traffic'].value_counts())
print(df['city_type'].value_counts())
print(df['order_time_of_day'].value_counts())
print(df['vehicle_condition'].value_counts())
print(df['multiple_deliveries'].value_counts())

low       15558
jam       14143
medium    10947
high       4425
Name: traffic, dtype: int64
metropolitian    34873
urban            10037
semi-urban         163
Name: city_type, dtype: int64
night         19607
evening        8758
morning        7718
late_night     4941
afternoon      4049
Name: order_time_of_day, dtype: int64
2    15034
1    15030
0    15009
Name: vehicle_condition, dtype: int64
1    28821
0    13937
2     1960
3      355
Name: multiple_deliveries, dtype: int64


In [12]:
traffic_order = ["low","medium","high","jam"]
city_type_order = ['semi-urban','urban','metropolitian']
order_time_of_day_order = ['morning','afternoon','evening','night','late_night']

In [13]:
# build a preprocessor

preprocessor = ColumnTransformer(transformers=[
    ("scale", MinMaxScaler(), num_cols),
    
    ("nominal_encode", OneHotEncoder(drop="first",handle_unknown="ignore",sparse_output=False), nominal_cat_cols),
    
    ("ordinal_encode", OrdinalEncoder(categories=[traffic_order,city_type_order,order_time_of_day_order]), ordinal_cat_cols)
     ],
    remainder="passthrough",n_jobs=-1,force_int_remainder_cols=False,verbose_feature_names_out=False)

preprocessor.set_output(transform="pandas")

,transformers,"[('scale', ...), ('nominal_encode', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,False
,feature_range,"(0, ...)"
,copy,True
,clip,False


In [14]:

X_train_trans = preprocessor.fit_transform(X_train)
X_test_trans = preprocessor.transform(X_test)

X_train_trans.head()

C:\Users\vkyad\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,age,ratings,pickup_time,distance,weather_fog,weather_sandstorms,weather_stormy,weather_sunny,weather_windy,type_of_order_drinks,type_of_order_meal,type_of_order_snack,type_of_vehicle_motorcycle,type_of_vehicle_scooter,festival_yes,city_name_ALH,city_name_AURG,city_name_BANG,city_name_BHP,city_name_CHEN,city_name_COIMB,city_name_DEH,city_name_GOA,city_name_HYD,city_name_INDO,city_name_JAP,city_name_KNP,city_name_KOC,city_name_KOL,city_name_LUDH,city_name_MUM,city_name_MYS,city_name_PUNE,city_name_RANCHI,city_name_SUR,city_name_VAD,order_day_2,order_day_3,order_day_4,order_day_5,order_day_6,order_day_7,order_day_8,order_day_9,order_day_10,order_day_11,order_day_12,order_day_13,order_day_14,order_day_15,order_day_16,order_day_17,order_day_18,order_day_19,order_day_20,order_day_21,order_day_23,order_day_24,order_day_25,order_day_26,order_day_27,order_day_28,order_day_29,order_day_30,order_day_31,order_month_3,order_month_4,order_day_of_week_Monday,order_day_of_week_Saturday,order_day_of_week_Sunday,order_day_of_week_Thursday,order_day_of_week_Tuesday,order_day_of_week_Wednesday,traffic,city_type,order_time_of_day,vehicle_condition,multiple_deliveries,is_weekend
28983,0.473684,0.92,1.0,0.234872,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1,1,0
18849,0.105263,0.96,0.5,0.163590,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2,1,0
28503,0.789474,0.88,0.0,0.163590,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,2,0,1
35742,0.578947,0.76,0.5,0.563590,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0,1,1
29645,0.789474,0.96,0.5,0.322564,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0,3.0,0,0,0


In [15]:
# transform target column

pt = PowerTransformer()

y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))

# Train Initial Baseline Model

In [16]:
from sklearn.linear_model import LinearRegression


In [17]:
lr = LinearRegression()

model_pipe = Pipeline(steps=[
                                ("preprocessing",preprocessor),
                                ("model",lr)
                            ])

model_pipe

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scale', ...), ('nominal_encode', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [18]:
model_pipe.fit(X_train,y_train_pt)

C:\Users\vkyad\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scale', ...), ('nominal_encode', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [19]:
y_pred_train = model_pipe.predict(X_train)
y_pred_test = model_pipe.predict(X_test)

In [20]:
# get the actual predictions values

y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

In [21]:
from sklearn.metrics import mean_absolute_error, r2_score

print(f"The train error is {mean_absolute_error(y_train,y_pred_train_org):.2f} minutes")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test_org):.2f} minutes")

The train error is 4.73 minutes
The test error is 4.71 minutes


In [22]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train_org):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test_org):.2f}")

The train r2 score is 0.60
The test r2 score is 0.61


In [23]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42,n_jobs=-1)

model_pipe = Pipeline(steps=[
                                ("preprocessing",preprocessor),
                                ("model",rf)
                            ])

model_pipe

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scale', ...), ('nominal_encode', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [24]:
model_pipe.fit(X_train,y_train_pt.ravel())

C:\Users\vkyad\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scale', ...), ('nominal_encode', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [25]:
y_pred_train = model_pipe.predict(X_train)
y_pred_test = model_pipe.predict(X_test)

In [26]:
y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

In [27]:
from sklearn.metrics import mean_absolute_error, r2_score

print(f"The train error is {mean_absolute_error(y_train,y_pred_train_org):.2f} minutes")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test_org):.2f} minutes")

The train error is 1.20 minutes
The test error is 3.24 minutes


In [28]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train_org):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test_org):.2f}")

The train r2 score is 0.97
The test r2 score is 0.81


In [29]:
 # Only works if set_output(transform="pandas") is used
feature_names = model_pipe.named_steps["preprocessing"].get_feature_names_out()
importances = model_pipe.named_steps["model"].feature_importances_
feature_importance = pd.Series(importances, index=feature_names).sort_values(ascending=False)

 feature_importance   

ratings                        0.206255
traffic                        0.148614
age                            0.115963
distance                       0.110658
vehicle_condition              0.084105
weather_sunny                  0.083605
multiple_deliveries            0.029541
weather_fog                    0.028109
weather_stormy                 0.017263
weather_sandstorms             0.017175
weather_windy                  0.016696
order_time_of_day              0.008600
pickup_time                    0.008037
festival_yes                   0.007459
city_type                      0.007433
type_of_order_drinks           0.003593
type_of_order_meal             0.003574
type_of_order_snack            0.003456
city_name_VAD                  0.003105
order_month_3                  0.002832
city_name_RANCHI               0.002483
city_name_MYS                  0.002460
city_name_SUR                  0.002454
city_name_PUNE                 0.002387
city_name_CHEN                 0.002363
